# EITQ TP4: Languages

## Credits

This tutorial is built around the examples from the `treelib` and `tatsu` documentations, as well as a [regexp](https://github.com/tesla809/intro-to-python-jupyter-notebooks/blob/master/47-Regular%20Expressions.ipynb) tutorial.  

## Python Versions


We'll be using Python 3. You can check your Python version at the command line by running `python --version`. In Colab, we can enforce the Python version by clicking `Runtime -> Change Runtime Type` and selecting `python3`.

In [1]:
!python --version

Python 3.10.8


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 20;

<IPython.core.display.Javascript object>

## Parsing

Regular expressions describe regular languages. Regular languages were discussed, at the theoretical-level during the lectures. They are an interesting, but nevertheless simple class of languages. 

The aim of this section is to teach you how to describe more complicated languages, such as programming languages. Programming languages were again discussed at the theoretical-level during the lectures, in terms of their grammar and semantics. 

This section will deal with language grammars, and semantics, at the practical level. We will rely on a library called `Tatsu` in order to :
1.   Describe language grammars.
2.   Recognize whether a given text obeys the grammar.
3.   *Parse* the text according to the grammar. 
4.   Provide semantics to the language.
5.   Translate the language into another. 

When a computer parses a text according to a grammar, it creates a tree-like data structure in memory, that represents the text in a well-organized manner. It is the computer's way of "understanding the text" and "keeping it in mind in an orderly fashion".  

When we provide a semantics for the language, we are telling the computer what to make of that tree-like representation of the text. For instance, if the language is progamming language, the tree represents a text which is a program, which the computer ought to execute. So, the semantics tells the computer how to take actions according to the content of that tree.

So, let us install the `Tatsu` library. Its documentation is [here](https://tatsu.readthedocs.io/en/stable/).

By-the-way, there are many other such libraries, nicely reviewed [here](https://tomassetti.me/parsing-in-python/). Another mainstream choice could have been the `Lark` library. 

In [3]:
!pip install ## Parsingtatsu

Defaulting to user installation because normal site-packages is not writeable
ERROR: You must give at least one requirement to install (see "pip help install")


Let us import the library so we can use it. We also take this opportunity to give a short nickname to some of the trees generated by `tatsu`. 

In [4]:
import tatsu
from tatsu.ast import AST
from tatsu.walkers import NodeWalker

from pprint import pprint
import json

## Mini-project



1.   By incremental modifications of the last version of the above-given grammars, provide a grammar for `Mini-ML` language we saw in class.
2.   Implement the big steps semantics of `Mini-ML` we saw in class. 

*Advanced.*

3.   In your grammar and semantics, modify the type of constants to be booleans instead of integers or floats, and let the primitives be Not, Or, And, Nand gates, so as to eventually obtain a custom language: `Circuit-ML`. 
4.    Implement a type-checker for `Circuit-ML`.


## Answser Q1

We simply implement the miniML grammar, starting from the grammars described in the TP.

In [5]:
miniMLGrammar = """
@@grammar::Calc


start
    =
    expression $
    ;


expression
    =
    | integer
    | first
    | second
    | function
    | apply
    | operator
    | pair
    | assign
    | subexpression
    | variable
    ;


integer::int
    =
    /-{0,1}\d+/
    ;

operator::Operator
    =
    operation:/\+|\-|\*|\//
    ;
    
variable::Variable
    =
    name:/[a-z]+/
    ;
    
pair::Pair
    =
    '(' first:expression ',' ~ second:expression ')'
    ;
    
first::First
    =
    'Fst' ~ parameter:expression
    ;
    
second::Second
    =
    'Snd' ~ parameter:expression
    ;
    
function::Lambda
    =
    'lambda' name:variable ':' ~ expression:expression
    ;

subexpression
    =
    '(' ~ @:expression ')'
    ;

assign::Assign
    =
    name:variable '=' ~ value:expression ';' ~ expression:expression
    ;
    
apply::Apply
    =
    function:expression ~ parameter:expression
    ;
"""

miniMLParser = tatsu.compile(miniMLGrammar, asmodel=True)

In [6]:
miniMlExamples = [
    "+ (-2, 3)",
    "/(Fst (2, 4), 1)",
    "a = 3; b = 4; *(a, b)",
    "(lambda a : (a, *(a, 4))) 2",
    "(lambda a : a (-2, 3)) +",
    "(lambda a : (a 5)) (lambda a : *(a, 2))",
    "(lambda a : (Snd a, (Fst a) (Snd a))) ((lambda a : *(a, 2)), 5)",
]

In [7]:
for example in miniMlExamples:
    if example != miniMlExamples[0]:
        print("\n"*3)
    ast = miniMLParser.parse(example)
    
    print("ast for \"" + example + "\":\n")
    print(json.dumps(ast.asjson(), indent=4))

ast for "+ (-2, 3)":

{
    "__class__": "Apply",
    "parameter": {
        "__class__": "Pair",
        "first": -2,
        "second": 3
    },
    "function": {
        "__class__": "Operator",
        "operation": "+"
    }
}




ast for "/(Fst (2, 4), 1)":

{
    "__class__": "Apply",
    "parameter": {
        "__class__": "Pair",
        "first": {
            "__class__": "First",
            "parameter": {
                "__class__": "Pair",
                "first": 2,
                "second": 4
            }
        },
        "second": 1
    },
    "function": {
        "__class__": "Operator",
        "operation": "/"
    }
}




ast for "a = 3; b = 4; *(a, b)":

{
    "__class__": "Assign",
    "value": 3,
    "name": {
        "__class__": "Variable",
        "name": "a"
    },
    "expression": {
        "__class__": "Assign",
        "value": 4,
        "name": {
            "__class__": "Variable",
            "name": "b"
        },
        "expression": {
          

## Answser Q2

We now need to implement a walker for the miniML grammar, although we have a few challenges to tackle:
 1) Functions: We assume that a function is always applied to something, so we can interprete "(lambda a : e1) e2" as "a = e1; e2".
 2) Operation: We handle operator as function, and operations as applying a function to an expression, this allows to pass an operator as a parameter.
 3) Variable: We need to handle *local* variable assignment "a = e1; e2". For that we populate a member dictionary similar to "variables = {a : e1}", then access it by "variables\[a\]" when encountering a varible (which is a type in the syntax). To keep this allocation local we delete/reassign to its previous value the key "a" in the dictionary after evaluating "e2" in "a = e1; e2".

In [8]:
class miniMLWalker(NodeWalker):
    def __init__(self, debug=False):
        self.debug = debug
        self.variables = {}
        
    def walk_object(self, node):
        if self.debug:
            print("Object", node)
            
        return node
    
    """
    functions for handling pairs
    """
    
    def walk__pair(self, node):
        if self.debug:
            print("Pair")
        
        return (self.walk(node.first), self.walk(node.second))
    
    def walk__first(self, node):
        if self.debug:
            print("First")
        
        parameter = self.walk(node.parameter)
        if isinstance(parameter, tuple):
            return self.walk(parameter[0])
        else:
            return self.walk(parameter.first)
    
    def walk__second(self, node):
        if self.debug:
            print("Second")
        
        parameter = self.walk(node.parameter)
        if isinstance(parameter, tuple):
            return self.walk(parameter[1])
        else:
            return self.walk(parameter.second) 
    
    """
    functions for handling variables
    """
    
    def assign_variable(self, variable_name, variable_value):
        # check if key was previously present, and save old value if so
        was_present = variable_name in self.variables
        old_value = self.variables[variable_name] if was_present else None
        # store new value for this key
        self.variables[variable_name] = variable_value
        return old_value
    def deassign_variable(self, variable_name, old_value):
        # store back original value, or remove the key
        if old_value is not None:
            self.variables[variable_name] = old_value
        else:
            del self.variables[variable_name]
    
    def walk__variable(self, node):
        if self.debug:
            print("Variable")
        
        if node in self.variables:
            return self.variables[node]
        else:
            return node
    
    def walk__assign(self, node):
        if self.debug:
            print("Assign")

        # assign variable
        old_value = self.assign_variable(node.name, self.walk(node.value))
        # compute expression
        results = self.walk(node.expression)
         # deassign variable
        self.deassign_variable(node.name, old_value)
            
        return results
    
    """
    functions for handling functions/operations
    """
    
    def walk__lambda(self, node):
        if self.debug:
            print("Lambda")
        
        return node
    
    def walk__apply(self, node):
        if self.debug:
            print("Apply")
            
        # get parameter value
        variable_value = self.walk(node.parameter)
        # read expression
        function = self.walk(node.function)
        
        """
        if the function is an operation then simply compute the value
        """
        if hasattr(function, "operation"):
            if function.operation == "+":
                return variable_value[0] + variable_value[1]
            elif function.operation == "-":
                return variable_value[0] - variable_value[1]
            elif function.operation == "*":
                return variable_value[0] * variable_value[1]
            elif function.operation == "/":
                return int(variable_value[0] / variable_value[1])
        
        """
        compute lambda like an assign
        replacing "(lambda x : e1) e2" with "x = e2; e1"
        """
        
        # assign variable
        old_value = self.assign_variable(function.name, variable_value)
        # compute expression
        results = self.walk(function.expression)
         # deassign variable
        self.deassign_variable(function.name, old_value)
        
        return results

In [9]:
"""
If you want to see debuging information, write:
debug = True
"""
debug = False

for example in miniMlExamples:
    ast = miniMLParser.parse(example)
    
    result = miniMLWalker(debug).walk(ast)
    print("The result of \"" + example + "\" is:", result)

The result of "+ (-2, 3)" is: 1
The result of "/(Fst (2, 4), 1)" is: 2
The result of "a = 3; b = 4; *(a, b)" is: 12
The result of "(lambda a : (a, *(a, 4))) 2" is: (2, 8)
The result of "(lambda a : a (-2, 3)) +" is: 1
The result of "(lambda a : (a 5)) (lambda a : *(a, 2))" is: 10
The result of "(lambda a : (Snd a, (Fst a) (Snd a))) ((lambda a : *(a, 2)), 5)" is: (5, 10)


## Answser Q3

There is no major challenges to implementing the circuitML grammar and walker starting from circuitML.

In [10]:
circuitMLGrammar = """
@@grammar::Calc


start
    =
    expression $
    ;

expression
    =
    | boolean
    | first
    | second
    | function
    | apply
    | operator
    | pair
    | assign
    | subexpression
    | variable
    | if_else
    ;


boolean::boolean
    =
    value:/True|False/
    ;

operator::Operator
    =
    operation:/&|\||!|Nand/
    ;
    
variable::Variable
    =
    name:/[a-z]+/
    ;
    
pair::Pair
    =
    '(' first:expression ',' ~ second:expression ')'
    ;
    
first::First
    =
    'Fst' ~ parameter:expression
    ;
    
second::Second
    =
    'Snd' ~ parameter:expression
    ;
    
function::Lambda
    =
    'lambda' name:variable ':' ~ expression:expression
    ;

subexpression
    =
    '(' ~ @:expression ')'
    ;
    
apply::Apply
    =
    function:expression ~ parameter:expression
    ;
    
assign::Assign
    =
    name:variable '=' ~ value:expression ';' ~ expression:expression
    ;

if_else::Ifelse
    =
    'If' condition:expression 'Then' ~ true:expression 'Else' ~ false:expression 'Endif'
    ;
"""

circuitMLParser = tatsu.compile(circuitMLGrammar, asmodel=True)

In [11]:
class circuitMLWalker(NodeWalker):
    def __init__(self, debug=False):
        self.debug = debug
        self.variables = {}
        
    def walk_object(self, node):
        if self.debug:
            print("Object", node)
            
        return node
    
    """
    functions for handling boolean
    """
    
    def walk_boolean(self, node):
        if self.debug:
            print("Boolean")
            
        return (node.value == "True")

    """
    functions for handling pairs
    """
    
    def walk__pair(self, node):
        if self.debug:
            print("Pair")
        
        return (self.walk(node.first), self.walk(node.second))
    
    def walk__first(self, node):
        if self.debug:
            print("First")
        
        parameter = self.walk(node.parameter)
        if isinstance(parameter, tuple):
            return self.walk(parameter[0])
        else:
            return self.walk(parameter.first)
    
    def walk__second(self, node):
        if self.debug:
            print("Second")
        
        parameter = self.walk(node.parameter)
        if isinstance(parameter, tuple):
            return self.walk(parameter[1])
        else:
            return self.walk(parameter.second)
    
    """
    functions for handling variables
    """
    
    def assign_variable(self, variable_name, variable_value):
        # check if key was previously present, and save old value if so
        was_present = variable_name in self.variables
        old_value = self.variables[variable_name] if was_present else None
        # store new value for this key
        self.variables[variable_name] = variable_value
        return old_value
    def deassign_variable(self, variable_name, old_value):
        # store back original value, or remove the key
        if old_value is not None:
            self.variables[variable_name] = old_value
        else:
            del self.variables[variable_name]
    
    def walk__variable(self, node):
        if self.debug:
            print("Variable")
        
        if node in self.variables:
            return self.variables[node]
        else:
            return node
    
    def walk__assign(self, node):
        if self.debug:
            print("Assign")

        # assign variable
        old_value = self.assign_variable(node.name, self.walk(node.value))
        # compute expression
        results = self.walk(node.expression)
         # deassign variable
        self.deassign_variable(node.name, old_value)
            
        return results
    
    """
    functions for handling functions
    """
    
    def walk__lambda(self, node):
        if self.debug:
            print("Lambda")
        
        return node
    
    def walk__apply(self, node):
        if self.debug:
            print("Apply")
            
        # get parameter value
        variable_value = self.walk(node.parameter)
        # read expression
        function = self.walk(node.function)
        
        """
        if the function is an operation then simply compute the value
        """
        if hasattr(function, "operation"):
            if function.operation == "&":
                return variable_value[0] and variable_value[1]
            elif function.operation == "|":
                return variable_value[0] or variable_value[1]
            elif function.operation == "!":
                return not variable_value
            elif function.operation == "Nand":
                return not (variable_value[0] and variable_value[1])
        
        """
        compute lambda like an assign
        replacing "(lambda x : e1) e2" with "x = e2; e1"
        """
        
        # assign variable
        old_value = self.assign_variable(function.name, variable_value)
        # compute expression
        results = self.walk(function.expression)
         # deassign variable
        self.deassign_variable(function.name, old_value)
        
        return results
    
    """
    functions for handling if elses
    """
    
    def walk__ifelse(self, node):
        if self.debug:
            print("Ifelse")
        
        if self.walk(node.condition):
            return self.walk(node.true)
        else:
            return self.walk(node.false)

Here are some example to prove that it works (we will not enable debugging for all examples as it is not understandable for bigger examples):

In [12]:
circuitMlExamples = [
    "! True",
    "Nand (True, False)",
    "a = True; (a, !a)",
    "If &(True, False) Then False Else True Endif",
    "(lambda a : (! a, |(a, !a))) True",
    "(lambda a : a False) !",
    "(lambda a : (a True)) (lambda a : !a)",
    "(lambda a : (Snd a, (Fst a) (Snd a))) ((lambda a : !a), True)",
]

In [13]:
for example in circuitMlExamples:
    if example != circuitMlExamples[0]:
        print("\n"*3)
    ast = circuitMLParser.parse(example)
    
    print("ast for \"" + example + "\":\n")
    print(json.dumps(ast.asjson(), indent=4))

ast for "! True":

{
    "__class__": "Apply",
    "parameter": {
        "__class__": "boolean",
        "value": "True"
    },
    "function": {
        "__class__": "Operator",
        "operation": "!"
    }
}




ast for "Nand (True, False)":

{
    "__class__": "Apply",
    "parameter": {
        "__class__": "Pair",
        "first": {
            "__class__": "boolean",
            "value": "True"
        },
        "second": {
            "__class__": "boolean",
            "value": "False"
        }
    },
    "function": {
        "__class__": "Operator",
        "operation": "Nand"
    }
}




ast for "a = True; (a, !a)":

{
    "__class__": "Assign",
    "value": {
        "__class__": "boolean",
        "value": "True"
    },
    "name": {
        "__class__": "Variable",
        "name": "a"
    },
    "expression": {
        "__class__": "Pair",
        "first": {
            "__class__": "Variable",
            "name": "a"
        },
        "second": {
            "__cla

In [14]:
"""
If you want to see debuging information, write:
debug = True
"""
debug = False

for example in circuitMlExamples:
    ast = circuitMLParser.parse(example)
    
    result = circuitMLWalker(debug).walk(ast)
    print("The result of \"" + example + "\" is:", result)

The result of "! True" is: False
The result of "Nand (True, False)" is: True
The result of "a = True; (a, !a)" is: (True, False)
The result of "If &(True, False) Then False Else True Endif" is: True
The result of "(lambda a : (! a, |(a, !a))) True" is: (False, True)
The result of "(lambda a : a False) !" is: True
The result of "(lambda a : (a True)) (lambda a : !a)" is: False
The result of "(lambda a : (Snd a, (Fst a) (Snd a))) ((lambda a : !a), True)" is: (True, False)


## Answser Q4

We add the requiered typing syntax, plus the anotation to the previously existing circuitML language.

NOTE: We don't add anotation to FST and SND as we assume that all pair type are known, either by deducing from the type of its two elements, or by the return type of a lambda (which are themselfs typed).

In [15]:
typedCircuitMLGrammar = """
@@grammar::Calc


start
    =
    | expression $
    | type
    ;

type
    =
    | booltype
    | pairtype
    | lambdatype
    | typesubexpression
    ;
    
typesubexpression
    =
    '(' ~ @:type ')'
    ;
    
booltype
    =
    'B'
    ;

pairtype::Pair
    =
    '(' first:type '*' ~ second:type ')'
    ;
    
lambdatype::Lambdatype
    =
    '(' variabletype:type '->' ~ resulttype:type ')'
    ;
    
expression
    =
    | boolean
    | first
    | second
    | function
    | apply
    | operator
    | pair
    | assign
    | subexpression
    | variable
    | if_else
    ;


boolean::boolean
    =
    value:/True|False/
    ;

operator::Operator
    =
    operation:/&|\||!|Nand/
    ;
    
variable::Variable
    =
    name:/[a-z]+/
    ;
    
pair::Pair
    =
    '(' first:expression ',' ~ second:expression ')'
    ;
    
first::First
    =
    'Fst' ~ parameter:expression
    ;
    
second::Second
    =
    'Snd' ~ parameter:expression
    ;
    
function::Lambda
    =
    'lambda' name:variable ~ '{' variabletype:type ~ '}' ':' ~ expression:expression
    ;

subexpression
    =
    '(' ~ @:expression ')'
    ;
    
apply::Apply
    =
    function:expression ~ parameter:expression
    ;
    
assign::Assign
    =
    name:variable '=' ~ value:expression ';' ~ expression:expression
    ;

if_else::Ifelse
    =
    'If' condition:expression 'Then' ~ true:expression 'Else' ~ false:expression 'Endif'
    ;
"""

typedCircuitMLParser = tatsu.compile(typedCircuitMLGrammar, asmodel=True)

We then need a walker to propagte the type of variables. This can mostly be done via modifying the existing walker.

The main chalenge is around lambda, which can either be a type by themselfs, or can be applyied.

To solve that, we access "variable type" of a lambda, and if it matches the type of the variable passed, we return the "return type" of the function.

In [16]:
class circuitMLTypeWalker(NodeWalker):
    def __init__(self, debug=False):
        self.debug = debug
        self.variables = {}
        
    def walk_object(self, node):
        if self.debug:
            print("Object", node)
            
        return "B"
    
    """
    functions for handling boolean
    """
    
    def walk_boolean(self, node):
        if self.debug:
            print("Boolean")
            
        return "B"

    """
    functions for handling pairs
    """
    
    def walk__pair(self, node):
        if self.debug:
            print("Pair")
        return "(" + self.walk(node.first) + "*" + self.walk(node.second) + ")"
    
    def walk__first(self, node):
        if self.debug:
            print("First")
        
        parameter = self.walk(node.parameter)
        parameter = typedCircuitMLParser.parse(parameter)
        return self.walk(parameter.first)
    
    def walk__second(self, node):
        if self.debug:
            print("Second")
        
        parameter = self.walk(node.parameter)
        parameter = typedCircuitMLParser.parse(parameter)
        return self.walk(parameter.second)
    
    """
    functions for handling variables
    """
    
    def assign_variable(self, variable_name, variable_value):
        # check if key was previously present, and save old value if so
        was_present = variable_name in self.variables
        old_value = self.variables[variable_name] if was_present else None
        # store new value for this key
        self.variables[variable_name] = variable_value
        return old_value
    def deassign_variable(self, variable_name, old_value):
        # store back original value, or remove the key
        if old_value is not None:
            self.variables[variable_name] = old_value
        else:
            del self.variables[variable_name]
    
    def walk__variable(self, node):
        if self.debug:
            print("Variable")
        
        if node in self.variables:
            return self.variables[node]
        else:
            return node
    
    def walk__assign(self, node):
        if self.debug:
            print("Assign")

        # assign variable
        old_value = self.assign_variable(node.name, self.walk(node.value))
        # compute expression
        results = self.walk(node.expression)
         # deassign variable
        self.deassign_variable(node.name, old_value)
            
        return results
    
    """
    functions for handling functions and operations
    """
    
    def walk__operator(self, node):
        if self.debug:
            print("Operator")
        
        if node.operation == "!":
            return "(B->B)"
        else:
            return "((B*B)->B)"
    
    def walk__lambdatype(self, node):
        if self.debug:
            print("Lambda")
            
        return "(" + self.walk(node.variabletype) + "->" + self.walk(node.resulttype) + ")"
    
    def walk__lambda(self, node):
        if self.debug:
            print("Lambda")
            
        # get parameter value
        variable_type = self.walk(node.variabletype)
        
        """
        compute lambda like an assign
        replacing "(lambda x : e1) e2" with "x = e2; e1"
        """
        
        # assign variable
        old_type = self.assign_variable(node.name, variable_type)
        # compute expression
        results = self.walk(node.expression)
        # deassign variable
        self.deassign_variable(node.name, old_type)
        
        return '(' + variable_type + "->" + results + ')'
    
    def walk__apply(self, node):
        if self.debug:
            print("Apply")
            
        # get parameter value
        variable_type = typedCircuitMLParser.parse(self.walk(node.parameter))
        # get function type
        function_type = typedCircuitMLParser.parse(self.walk(node.function))
        
        assert function_type.variabletype == variable_type, "variable type missmatch in lambda !"
        return self.walk(function_type.resulttype)
    
    """
    functions for handling if elses
    """
    
    def walk__ifelse(self, node):
        if self.debug:
            print("Ifelse")
            
        assert self.walk(node.condition) == "B", "Non boolean parameter missmatch in if !"
        
        if self.walk(node.condition):
            return self.walk(node.true)
        else:
            return self.walk(node.false)

Here are some example to prove that it works (we will not enable debugging for all examples as it is not understandable for bigger examples):

In [17]:
circuitMlTypeExamples = [
    "! True",
    "Nand",
    "!",
    "& (True, False)",
    "a = True; (a, !a)",
    "If &(True, False) Then False Else True Endif",
    "(lambda a {B} : (! a, |(a, !a))) True",
    "(lambda a {(B -> B)} : a True) !",
    "lambda a {((B -> B) * B)} : (Snd a, (Fst a) (Snd a))",
]

In [18]:
for example in circuitMlTypeExamples:
    if example != circuitMlTypeExamples[0]:
        print("\n"*3)
    ast = typedCircuitMLParser.parse(example)
    
    print("ast for \"" + example + "\":\n")
    print(json.dumps(ast.asjson(), indent=4))

ast for "! True":

{
    "__class__": "Apply",
    "parameter": {
        "__class__": "boolean",
        "value": "True"
    },
    "function": {
        "__class__": "Operator",
        "operation": "!"
    }
}




ast for "Nand":

{
    "__class__": "Operator",
    "operation": "Nand"
}




ast for "!":

{
    "__class__": "Operator",
    "operation": "!"
}




ast for "& (True, False)":

{
    "__class__": "Apply",
    "parameter": {
        "__class__": "Pair",
        "first": {
            "__class__": "boolean",
            "value": "True"
        },
        "second": {
            "__class__": "boolean",
            "value": "False"
        }
    },
    "function": {
        "__class__": "Operator",
        "operation": "&"
    }
}




ast for "a = True; (a, !a)":

{
    "__class__": "Assign",
    "value": {
        "__class__": "boolean",
        "value": "True"
    },
    "name": {
        "__class__": "Variable",
        "name": "a"
    },
    "expression": {
        "__cla

In [19]:
"""
If you want to see debuging information, write:
debug = True
"""
debug = False

for example in circuitMlTypeExamples:
    ast = typedCircuitMLParser.parse(example)
    
    result = circuitMLTypeWalker(debug).walk(ast)
    print("The type of \"" + example + "\" is: \"" + str(result) + "\"")

The type of "! True" is: "B"
The type of "Nand" is: "((B*B)->B)"
The type of "!" is: "(B->B)"
The type of "& (True, False)" is: "B"
The type of "a = True; (a, !a)" is: "(B*B)"
The type of "If &(True, False) Then False Else True Endif" is: "B"
The type of "(lambda a {B} : (! a, |(a, !a))) True" is: "(B*B)"
The type of "(lambda a {(B -> B)} : a True) !" is: "B"
The type of "lambda a {((B -> B) * B)} : (Snd a, (Fst a) (Snd a))" is: "(((B->B)*B)->(B*B))"
